# 💧 Humidity — Anomaly Detection & Forecasting

This notebook loads humidity data from `temp_hum.csv`, performs EDA, trains an **Isolation Forest** anomaly detector and a **Linear Regression** forecaster (using temperature as a feature), then exports models.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.ensemble import IsolationForest
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, mean_squared_error
import joblib, os, warnings
warnings.filterwarnings('ignore')
sns.set_style('whitegrid')
print('Libraries loaded ✅')

## 1 — Load & Clean Data

In [ ]:
raw = pd.read_csv('../raw/temp_hum.csv', header=None, names=['date', 'time', 'temp_raw', 'hum_raw'])
print(f'Raw rows: {len(raw)}')

# Filter error rows
raw = raw[raw['temp_raw'].str.startswith('T=', na=False)].copy()

# Parse values
raw['temperature'] = raw['temp_raw'].str.replace('T=', '', regex=False).astype(float)
raw['humidity'] = raw['hum_raw'].str.replace('H=', '', regex=False).astype(float)

# Build datetime index
raw['datetime'] = pd.to_datetime(raw['date'] + ' ' + raw['time'], format='%m/%d/%y %H:%M:%S')
df = raw[['datetime', 'temperature', 'humidity']].set_index('datetime').sort_index()

# Remove extreme outliers
df = df[(df['humidity'] > 5) & (df['humidity'] < 100) & (df['temperature'] > 10)]
print(f'Clean rows: {len(df)}')
df.describe()

## 2 — Exploratory Data Analysis

In [ ]:
fig, axes = plt.subplots(2, 2, figsize=(14, 8))

# Time-series
axes[0, 0].plot(df.index, df['humidity'], color='#3498db', linewidth=0.8)
axes[0, 0].set_title('Humidity Over Time')
axes[0, 0].set_ylabel('%RH')

# Distribution
axes[0, 1].hist(df['humidity'], bins=20, color='#3498db', edgecolor='white', alpha=0.8)
axes[0, 1].set_title('Humidity Distribution')
axes[0, 1].set_xlabel('%RH')

# Correlation with temperature
axes[1, 0].scatter(df['temperature'], df['humidity'], alpha=0.5, color='#2ecc71', s=15)
axes[1, 0].set_title('Humidity vs Temperature')
axes[1, 0].set_xlabel('Temperature (°C)')
axes[1, 0].set_ylabel('Humidity (%RH)')

# Boxplot by hour
df['hour'] = df.index.hour
df.boxplot(column='humidity', by='hour', ax=axes[1, 1])
axes[1, 1].set_title('Humidity by Hour of Day')
axes[1, 1].set_xlabel('Hour')
axes[1, 1].set_ylabel('%RH')
plt.suptitle('')

plt.tight_layout()
plt.show()

print(f"\nCorrelation (Temp ↔ Humidity): {df['temperature'].corr(df['humidity']):.3f}")

## 3 — Feature Engineering

In [ ]:
df['hour'] = df.index.hour
df['minute'] = df.index.minute
df['hum_rolling_mean'] = df['humidity'].rolling(window=6, min_periods=1).mean()
df['hum_rolling_std'] = df['humidity'].rolling(window=6, min_periods=1).std().fillna(0)
df['hum_lag_1'] = df['humidity'].shift(1).bfill()
df['hum_lag_3'] = df['humidity'].shift(3).bfill()

features = ['hour', 'minute', 'temperature', 'hum_rolling_mean', 'hum_rolling_std', 'hum_lag_1', 'hum_lag_3']
df.dropna(inplace=True)
print(f'Feature-engineered rows: {len(df)}')
df[features].head()

## 4 — Anomaly Detection (Isolation Forest)

In [ ]:
iso_model = IsolationForest(contamination=0.05, random_state=42, n_estimators=100)
df['anomaly'] = iso_model.fit_predict(df[['humidity']])
anomalies = df[df['anomaly'] == -1]

plt.figure(figsize=(14, 4))
plt.plot(df.index, df['humidity'], label='Normal', color='#3498db', linewidth=0.8)
plt.scatter(anomalies.index, anomalies['humidity'], color='red', s=40, zorder=5, label=f'Anomalies ({len(anomalies)})')
plt.title('Humidity Anomaly Detection (Isolation Forest)')
plt.ylabel('%RH')
plt.legend()
plt.tight_layout()
plt.show()

print(f'Anomalies detected: {len(anomalies)} / {len(df)} ({100*len(anomalies)/len(df):.1f}%)')

## 5 — Humidity Forecasting (Linear Regression)

In [ ]:
X = df[features]
y = df['humidity']

split = int(len(df) * 0.8)
X_train, X_test = X.iloc[:split], X.iloc[split:]
y_train, y_test = y.iloc[:split], y.iloc[split:]

reg_model = LinearRegression()
reg_model.fit(X_train, y_train)
y_pred = reg_model.predict(X_test)

mae = mean_absolute_error(y_test, y_pred)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
print(f'MAE:  {mae:.2f} %RH')
print(f'RMSE: {rmse:.2f} %RH')

plt.figure(figsize=(14, 4))
plt.plot(y_test.index, y_test.values, label='Actual', color='#2c3e50')
plt.plot(y_test.index, y_pred, label='Predicted', color='#3498db', linestyle='--')
plt.title(f'Humidity Forecast — MAE={mae:.2f}, RMSE={rmse:.2f}')
plt.ylabel('%RH')
plt.legend()
plt.tight_layout()
plt.show()

## 6 — Export Models

In [ ]:
os.makedirs('../models', exist_ok=True)

joblib.dump({
    'isolation_forest': iso_model,
    'linear_regression': reg_model,
    'features': features,
    'metrics': {'mae': mae, 'rmse': rmse}
}, '../models/humidity_model.pkl')

print('✅ Model exported to models/humidity_model.pkl')